합성곱 신경망(Convolution Neural Networks, CNN)

   - 완전 연결 네트워크의 문제점으로부터 시작
     - 매개변수의 폭발적인 증가
     - 공간 추론의 부족, 픽셀 사이의 근접성 개념이 완전 연결 계층(Fully-Connected Layer)에서는 손실됨
   - 합성곱 계층은 입력 이미지가 커져도 튜닝해야 할 매개변수 개수에 영향을 주지 않음
   - 어떠한 이미지에도 그 차원 수와 상관없이 적용될 수 있음

In [1]:
# tensorflow에서 직접 CNN 함수구현 (저수준의 API)

import tensorflow as tf
k, D, N = (3, 16, 32)

kernel_size = [k, k, D, N]
glorot_uni_initializer = tf.initializers.GlorotUniform()

kernels = tf.Variable(glorot_uni_initializer(kernel_size),
                      trainable=True, name='filters')
bias = tf.Variable(tf.zeros(shape=[N]), trainable=True, name='bias')

@tf.function
def conv_layer(x, kernels, bias, s):
    z = tf.nn.conv2d(x, kernels, strides=[1, s, s, 1], padding='VALID')
    return tf.nn.relu(z + bias)

#  Class 만들기
class SimpleCNN(tf.keras.layers.Layer):
    def __init__(self, num_kernels=32, kernel_size=(3, 3), strides=1):
        super().__init__()
        self.num_kernels = num_kernels
        self.kernel_size = kernel_size
        self.strides = strides
    def build(self, input_shape):
        input_channels = input_shape[-1]
        
        kernels_shape = (*self.kernel_size, input_channels, self.num_kernels)
        glorot_init = tf.initializers.GlorotUniform()
        self.kernels = self.add_weight(neme='kernels', shape=kernels_shape, 
                                       initializer=glorot_init, trainable=True)
        
        self.bias = self.add_weight(name='bias', shape=(self.num_kernels, ),
                                    initializer='random_normal', trainable=True)
    def call(self, inputs):
        return conv_layer(inputs, self.kernels, self.bias, self.strides)

유효 수용 영역(ERF, Effective Receptive Field)
  - 입력 이미지에서 거리가 먼 요소를 상호 참조하여 결합, 네트워크 능력에 영향을 줌
  - 입력 이미지의 영역을 정의해 주어진 계층을 위한 뉴런의 활성화에 영향을 미침
  - 한 계층의 필터 크기나 윈도우 크기로 불리기 때문에 RF(receptive field, 수용 영역)이라는 용어를 흔히 볼 수 있음

CNN의 원조, LeNet-5 구현하기

In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.datasets import mnist

import numpy as np

In [3]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

# scaler 
x_train, y_train = x_train/255.0, y_train/255.0
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [4]:
num_classes = 10
epochs = 100
batch_size = 32

In [5]:
class LeNet5(Model):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = Conv2D(6, kernel_size=(5, 5), padding='same', activation='relu')
        self.conv2 = Conv2D(16, kernel_size=(5, 5), activation='relu')
        self.max_pool = MaxPool2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, input_data):
        x = self.max_pool(self.conv1(input_data))
        x = self.max_pool(self.conv2(x))
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x)))
        
        return x
    

# 모델
model = LeNet5(num_classes)

model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss'), tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)]

model.fit(x_train, y_train, 
          batch_size=batch_size, 
          epochs=epochs,
          validation_data=(x_test, y_test), 
          callbacks=callbacks)

Epoch 1/100
1863/1875 [============================>.] - ETA: 0s - loss: 0.0140 - accuracy: 0.0986

TypeError: in user code:

    File "C:\ProgramData\Anaconda3\envs\tf270\lib\site-packages\keras\engine\training.py", line 1366, in test_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\tf270\lib\site-packages\keras\engine\training.py", line 1356, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\tf270\lib\site-packages\keras\engine\training.py", line 1349, in run_step  **
        outputs = model.test_step(data)
    File "C:\ProgramData\Anaconda3\envs\tf270\lib\site-packages\keras\engine\training.py", line 1303, in test_step
        y_pred = self(x, training=False)
    File "C:\ProgramData\Anaconda3\envs\tf270\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    TypeError: Exception encountered when calling layer "le_net5" (type LeNet5).
    
    in user code:
    
        Fil